In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql as pymysql
from urllib.parse import quote
import requests
import json

In [2]:
ZABBIX_API_URL = "http://192.168.10.109/zabbix/api_jsonrpc.php"
UNAME = "Admin"
PWORD = "zabbix"

In [ ]:
try:
    engine = create_engine('mysql+pymysql://softnix:%s@192.168.10.121/zabbix' % quote('softnix'))
except:
    print('Database Username or Password went wrong')

In [ ]:
readFile = pd.read_csv('./users.csv',sep=";")

In [3]:
def Auth():
    r = requests.post(ZABBIX_API_URL,
                  json={
                      "jsonrpc": "2.0",
                      "method": "user.login",
                      "params": {
                          "user": UNAME,
                          "password": PWORD},
                      "id": 1
                  })
    AUTHTOKEN = r.json()["result"]
    return AUTHTOKEN

In [ ]:
def group_id(group):
    r = requests.post(ZABBIX_API_URL, json={
        "jsonrpc": "2.0",
            "method": "usergroup.get",
            "params": {
                "filter": {
                    "name": [
                        str(group)
                    ]
                },
                "output": "extend"
            },
            "auth": Auth(),
            "id": 1
    })
    strID = ''
    for i in r.json()['result']:
        strID = i['usrgrpid']
    return strID

In [ ]:
def role_id(role):
    r = requests.post(ZABBIX_API_URL, json={
        "jsonrpc": "2.0",
        "method": "role.get",
        "params": {
            "filter": {
                "name": [
                    str(role)
                ]
            },
            "output": "extend"
        },
        "auth": Auth(),
        "id": 1
    })
    strID = ''
    for i in r.json()['result']:
        strID = i['roleid']
    return strID

In [26]:
def user_get(user):
    r = requests.post(ZABBIX_API_URL, json={
        "jsonrpc": "2.0",
        "method": "user.get",
        "params": {
            "filter": {
                "username": [
                    str(user)
                ]
            },
            "output": "extends"
        },
        "auth": Auth(),
        "id": 1
    })
    strID = ''
    for i in r.json()['result']:
        strID = i['userid']
    return strID

In [6]:
def media_get(media):
    r = requests.post(ZABBIX_API_URL, json={
    "jsonrpc": "2.0",
    "method": "mediatype.get",
    "params": {
        "filter": {
            "name": [
                str(media)
            ]
        },
        "output": "extend",
        "selectMessageTemplates": "extend"
    },
    "auth": Auth(),
    "id": 1
    })
    strID = ''
    for i in r.json()['result']:
        strID = i['mediatypeid']
    return strID

SyntaxError: invalid syntax (4291406408.py, line 14)

In [ ]:
def create_user(login,name,surname,groupid,password,roleid, media, email):
    r = requests.post(ZABBIX_API_URL, json={
        "jsonrpc": "2.0",
        "method": "user.create",
        "params": {
            "username": str(login),
            "passwd": str(password),
            "roleid": str(roleid),
            "usrgrps": [
                {
                    "usrgrpid": str(groupid)
                }
            ],
            "medias": [
                {
                    "mediatypeid": str(media),
                    "sendto": [
                        str(email)
                    ],
                    "active": 0,
                    "severity": 63,
                    "period": "1-7,00:00-24:00"
                }
            ]
        },
        "auth": Auth(),
        "id": 1
    })
    for i in r.json():
        if 'result' in i:
            return i['result']
        elif 'error' in i:
            return i['error']['data']

In [ ]:
def last_mediaid():
    query = engine.execute("SELECT MAX(mediaid)+1 as maxis FROM media LIMIT 1")
    myresult = query.fetchone()[0]
    if myresult > 0:
        return myresult
    else:
        myresult+1

In [ ]:
def last_groupid():
    query = engine.execute("SELECT MAX(id)+1 as maxid FROM users_groups LIMIT 1")
    myresult = query.fetchone()[0]
    if myresult > 0:
        return myresult
    else:
        return myresult+1

In [ ]:
def insert_user():
    for index, i in readFile.iterrows():
        index = index +1
        groupid = group_id(i['group'].split(",")[0])
        roleid = role_id(i['role'])
        media_id = media_get(i['mediatype'])
        if len(i['group']) > 0:
            try:
                create_user(i['login'], i['name'], i['surname'], groupid, i['pass'], roleid, media_id, i['email'])
            except Exception as e:
                print('Error: [',i['login'],'] Ready Existes')
                continue
        else:
            print('Group is empty in file CSV')
        print(index, 'Add Users: -> [',i['login'],'] Successfully')

In [ ]:
def assign_group():
    group_list = pd.DataFrame()
    group_list['login'] = readFile['login']
    group_list['group'] = readFile['group'].str.split(',')
    df_result = group_list.set_index(['login']).apply(pd.Series.explode).reset_index()
    return df_result

In [ ]:
def insert_usrgrp():
    for i,x in assign_group().iterrows():
        try:
            max_id = last_groupid()
            groupid = group_id(x['group'])
            userid = user_get(x['login'])
            if len(x['group']) > 0:
                sql = f'''
                        INSERT IGNORE INTO
                            users_groups
                        SET
                            id = {max_id},
                            usrgrpid = {groupid},
                            userid = {userid}
                    '''
                engine.execute(sql)
            else:
                print('Group is empty in file CSV.')
        except IndexError:
                 print('--User Group not match in Database Please Insert Usergroup in Zabbix Web Before Running Script !.--')
                 continue
        print(f'Add More Group ->[',x['group'],']','->[', x['login'], '] Successfully')

In [ ]:
#เรียกใช้ Function
if __name__ == '__main__':
    print('----------INSERT USERS-----------')
    insert_user()
    print('----------INSERT USER GROUP-----------')
    insert_usrgrp()
    print('----------SUCCESSFULLY-----------')